In [ ]:
import kagglehub
import pandas as pd
import os
import chardet

# Detecta a codificação de um arquivo
def detectar_codificacao(caminho_arquivo):
    with open(caminho_arquivo, 'rb') as f:
        resultado = chardet.detect(f.read(10000))
    return resultado['encoding']

# Converte arquivos CSV de um diretório para Parquet
def converter_csv_para_parquet(diretorio_csv, diretorio_saida):
    os.makedirs(diretorio_saida, exist_ok=True)

    for arquivo in os.listdir(diretorio_csv):
        if arquivo.endswith('.csv'):
            caminho_csv = os.path.join(diretorio_csv, arquivo)
            nome_base = os.path.splitext(arquivo)[0]
            caminho_parquet = os.path.join(diretorio_saida, nome_base + '.parquet')

            try:
                codificacao = detectar_codificacao(caminho_csv)

                try:
                    df = pd.read_csv(caminho_csv, encoding=codificacao)
                except UnicodeDecodeError:
                    df = pd.read_csv(caminho_csv, encoding='latin1')

                df.to_parquet(caminho_parquet, index=False, engine='pyarrow')

            except Exception as e:
                print(f"Erro ao processar {arquivo}: {e}")

# Download dos datasets
path1 = kagglehub.dataset_download("michau96/one-piece-characters-and-chapters")
path2 = kagglehub.dataset_download("greninja1729/one-piece-imdb-ratings-and-votes")

# Diretório de saída
saida_dir = os.path.join(os.getcwd(), "datasets_parquet")

# Conversão
converter_csv_para_parquet(path1, saida_dir)
converter_csv_para_parquet(path2, saida_dir)

100%|██████████| 52.3k/52.3k [00:00<00:00, 30.4MB/s]

Extracting files...


100%|██████████| 123k/123k [00:00<00:00, 50.1MB/s]

Extracting files...


In [ ]:
from bs4 import BeautifulSoup
import requests
import csv

# URL da página com os volumes
url = "https://onepiece.fandom.com/pt/wiki/Cap%C3%ADtulos_e_Volumes/Volumes"
headers = {"User-Agent": "Mozilla/5.0"}

# Fazendo o request
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

# Lista para guardar os dados
volumes = []

# Encontrar todas as seções que contêm os volumes
for table in soup.select("table.collapsible"):
    span_tag = table.select_one("th span")
    img_tag = table.select_one("img")

    # Pula se não tiver título ou imagem
    if not span_tag or not img_tag:
        continue

    volume_nome = span_tag.text.strip()
    img_url = img_tag.get("data-src", "")

    if img_url.startswith("https://static.wikia.nocookie.net"):
        volumes.append([volume_nome, img_url])

# Salvando em CSV
with open("capas_volumes_onepiece.csv", mode="w", newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["volume", "img_url"])
    writer.writerows(volumes)

print("CSV gerado com sucesso!")

CSV gerado com sucesso!
